In [1]:
{
  "cells": [
    {
      "cell_type": "markdown",
      "metadata": {},
      "source": [
        "# 📈 News-Aktien-Analyse mit Gemini API\n",
        "\n",
        "Dieses Notebook analysiert Nachrichtenartikel und deren Auswirkungen auf Aktienkurse.\n",
        "\n",
        "**Features:**\n",
        "- ✅ Lädt News und Aktiendaten aus CSV\n",
        "- ✅ Nutzt Gemini API für Artikel-Analyse UND Webseiten-Extraktion\n",
        "- ✅ Identifiziert betroffene Aktien und Zeitfenster\n",
        "- ✅ Visualisiert Kursentwicklung im Kontext\n",
        "- ✅ Exportiert Ergebnisse (CSV, JSON, PNG)"
      ]
    },
    {
      "cell_type": "markdown",
      "metadata": {},
      "source": [
        "## 1️⃣ Installation & Imports"
      ]
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {},
      "outputs": [],
      "source": [
        "# Installiere benötigte Pakete\n",
        "!pip install -q google-generativeai pandas matplotlib numpy"
      ]
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {},
      "outputs": [],
      "source": [
        "import pandas as pd\n",
        "import numpy as np\n",
        "import json\n",
        "import google.generativeai as genai\n",
        "from datetime import datetime, timedelta\n",
        "import matplotlib.pyplot as plt\n",
        "import matplotlib.dates as mdates\n",
        "from pathlib import Path\n",
        "import time\n",
        "import re\n",
        "from IPython.display import display, Image, Markdown\n",
        "\n",
        "print(\"✅ Alle Bibliotheken importiert!\")"
      ]
    },
    {
      "cell_type": "markdown",
      "metadata": {},
      "source": [
        "## 2️⃣ Konfiguration"
      ]
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {},
      "outputs": [],
      "source": [
        "# ============================================================================\n",
        "# WICHTIG: Hier deinen Gemini API Key einfügen!\n",
        "# ============================================================================\n",
        "GEMINI_API_KEY = \"DEIN_GEMINI_API_KEY_HIER\"\n",
        "\n",
        "# Gemini konfigurieren\n",
        "genai.configure(api_key=GEMINI_API_KEY)\n",
        "\n",
        "# Model wählen (gemini-1.5-flash ist schnell und günstig, gemini-1.5-pro ist präziser)\n",
        "MODEL_NAME = \"gemini-1.5-flash\"\n",
        "model = genai.GenerativeModel(MODEL_NAME)\n",
        "\n",
        "print(f\"✅ Gemini API konfiguriert: {MODEL_NAME}\")"
      ]
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {},
      "outputs": [],
      "source": [
        "# Dateipfade\n",
        "NEWS_FILE = \"data/news_6m_finnhub_newsapi.csv\"\n",
        "STOCK_FILES = {\n",
        "    \"NVDA\": \"Stockcorse_as_csv/NVDA_6monatealles.csv\",\n",
        "    \"TSLA\": \"Stockcorse_as_csv/TSLA_6monatealles.csv\",\n",
        "    \"ASML\": \"Stockcorse_as_csv/ASML_6monatealles.csv\",\n",
        "    \"META\": \"Stockcorse_as_csv/META_6monatealles.csv\",\n",
        "    \"AMZN\": \"Stockcorse_as_csv/AMZN_6monatealles.csv\"\n",
        "}\n",
        "\n",
        "# Output-Ordner\n",
        "OUTPUT_DIR = Path(\"analysis_results\")\n",
        "OUTPUT_DIR.mkdir(exist_ok=True)\n",
        "\n",
        "# Anzahl Artikel zum Analysieren\n",
        "MAX_ARTICLES = 20\n",
        "\n",
        "print(f\"📁 Output-Ordner: {OUTPUT_DIR}\")\n",
        "print(f\"📰 Analysiere {MAX_ARTICLES} Artikel\")"
      ]
    },
    {
      "cell_type": "markdown",
      "metadata": {},
      "source": [
        "## 3️⃣ Daten laden"
      ]
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {},
      "outputs": [],
      "source": [
        "def load_data():\n",
        "    \"\"\"Lädt alle CSV-Dateien\"\"\"\n",
        "    print(\"📊 Lade Daten...\\n\")\n",
        "    \n",
        "    # News laden\n",
        "    news_df = pd.read_csv(NEWS_FILE)\n",
        "    print(f\"✓ News geladen: {len(news_df)} Artikel\")\n",
        "    print(f\"  Spalten: {list(news_df.columns)}\\n\")\n",
        "    \n",
        "    # Aktiendaten laden\n",
        "    stocks_data = {}\n",
        "    for ticker, filepath in STOCK_FILES.items():\n",
        "        df = pd.read_csv(filepath)\n",
        "        \n",
        "        # Versuche Date-Spalte zu finden\n",
        "        date_col = 'Date' if 'Date' in df.columns else df.columns[0]\n",
        "        df['Date'] = pd.to_datetime(df[date_col])\n",
        "        df = df.sort_values('Date')\n",
        "        \n",
        "        stocks_data[ticker] = df\n",
        "        print(f\"✓ {ticker}: {len(df)} Handelstage ({df['Date'].min().date()} bis {df['Date'].max().date()})\")\n",
        "    \n",
        "    return news_df, stocks_data\n",
        "\n",
        "# Daten laden\n",
        "news_df, stocks_data = load_data()"
      ]
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {},
      "outputs": [],
      "source": [
        "# Vorschau der News-Daten\n",
        "print(\"📰 Erste 3 News-Artikel:\\n\")\n",
        "display(news_df.head(3))"
      ]
    },
    {
      "cell_type": "markdown",
      "metadata": {},
      "source": [
        "## 4️⃣ Gemini-Funktionen"
      ]
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {},
      "outputs": [],
      "source": [
        "def extract_article_with_gemini(url, headline, summary):\n",
        "    \"\"\"\n",
        "    Nutzt Gemini um den vollständigen Artikel-Inhalt von einer URL zu extrahieren.\n",
        "    Gemini kann URLs direkt verarbeiten!\n",
        "    \"\"\"\n",
        "    prompt = f\"\"\"Bitte lies den Artikel unter dieser URL und extrahiere den vollständigen Haupttext (ohne Werbung, Navigation, etc.).\n",
        "\n",
        "URL: {url}\n",
        "\n",
        "Als Kontext:\n",
        "Headline: {headline}\n",
        "Summary: {summary}\n",
        "\n",
        "Gib nur den reinen Artikeltext zurück, maximal 3000 Wörter.\"\"\"\n",
        "    \n",
        "    try:\n",
        "        response = model.generate_content(prompt)\n",
        "        \n",
        "        if response and response.text:\n",
        "            return response.text\n",
        "        else:\n",
        "            print(f\"  ⚠ Gemini konnte URL nicht verarbeiten: {url}\")\n",
        "            return summary  # Fallback auf Summary\n",
        "            \n",
        "    except Exception as e:\n",
        "        print(f\"  ⚠ Fehler beim Extrahieren: {e}\")\n",
        "        return summary  # Fallback auf Summary\n",
        "\n",
        "print(\"✅ extract_article_with_gemini() definiert\")"
      ]
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {},
      "outputs": [],
      "source": [
        "def analyze_article_with_gemini(headline, summary, full_content, published_date):\n",
        "    \"\"\"\n",
        "    Analysiert Artikel mit Gemini und extrahiert strukturierte Informationen:\n",
        "    - Betroffene Aktie (NVDA, TSLA, ASML, META, AMZN)\n",
        "    - Zeitfenster des Einflusses (z.B. 0-3d, -1d-5d, 0-6h)\n",
        "    - Begründung\n",
        "    - Sentiment\n",
        "    \"\"\"\n",
        "    \n",
        "    prompt = f\"\"\"Du bist ein Finanzanalyst. Analysiere diesen Nachrichtenartikel und bestimme:\n",
        "\n",
        "1. BETROFFENE AKTIE: Welches Unternehmen ist hauptsächlich betroffen? \n",
        "   Wähle GENAU EINES: NVDA, TSLA, ASML, META, oder AMZN. \n",
        "   Falls keines zutrifft, sage \"NONE\".\n",
        "\n",
        "2. ZEITFENSTER: Bestimme das Zeitfenster des Einflusses auf den Aktienkurs. \n",
        "   Formate (Beispiele):\n",
        "   - \"0-3d\" = 0 bis 3 Tage nach Veröffentlichung\n",
        "   - \"-1d-5d\" = 1 Tag VOR bis 5 Tage NACH Veröffentlichung\n",
        "   - \"0-30d\" = 0 bis 30 Tage nach Veröffentlichung\n",
        "   - \"0-90d\" = bis Quartalsende (ca. 90 Tage)\n",
        "   - \"0-6h\" = 0 bis 6 Stunden nach Veröffentlichung\n",
        "   - \"0-2h\" = 0 bis 2 Stunden (für breaking news)\n",
        "   \n",
        "   Denke logisch:\n",
        "   - Earnings Reports = kurze Wirkung (0-3d)\n",
        "   - Produktankündigungen = mittelfristig (0-14d)\n",
        "   - Regulatorische Änderungen = langfristig (0-90d)\n",
        "   - Breaking News = sehr kurz (0-6h)\n",
        "   - Strategische Partnerschaften = mittelfristig (0-30d)\n",
        "\n",
        "3. BEGRÜNDUNG: Kurze Erklärung (2-3 Sätze) warum dieses Zeitfenster gewählt wurde.\n",
        "   Beziehe dich auf konkrete Inhalte des Artikels.\n",
        "\n",
        "4. SENTIMENT: Wähle: POSITIVE, NEGATIVE, oder NEUTRAL\n",
        "\n",
        "ARTIKEL-DATEN:\n",
        "Veröffentlicht: {published_date}\n",
        "Headline: {headline}\n",
        "Summary: {summary}\n",
        "Volltext (Auszug): {full_content[:4000]}\n",
        "\n",
        "ANTWORTE IM EXAKTEN JSON-FORMAT (NUR JSON, keine Markdown-Blöcke!):\n",
        "{{\n",
        "  \"ticker\": \"NVDA\",\n",
        "  \"timeframe\": \"0-3d\",\n",
        "  \"reasoning\": \"Deine Begründung hier\",\n",
        "  \"sentiment\": \"POSITIVE\"\n",
        "}}\n",
        "\n",
        "Gib NUR das JSON zurück, keine zusätzlichen Erklärungen!\"\"\"\n",
        "\n",
        "    try:\n",
        "        response = model.generate_content(prompt)\n",
        "        \n",
        "        if not response or not response.text:\n",
        "            return None\n",
        "        \n",
        "        # Extrahiere JSON aus der Antwort\n",
        "        text = response.text.strip()\n",
        "        \n",
        "        # Entferne Markdown-Blöcke falls vorhanden\n",
        "        text = re.sub(r'```json\\s*', '', text)\n",
        "        text = re.sub(r'```\\s*', '', text)\n",
        "        \n",
        "        # Finde JSON-Objekt\n",
        "        json_match = re.search(r'\\{.*\\}', text, re.DOTALL)\n",
        "        if json_match:\n",
        "            json_str = json_match.group(0)\n",
        "            analysis = json.loads(json_str)\n",
        "            \n",
        "            # Validierung\n",
        "            valid_tickers = list(STOCK_FILES.keys()) + ['NONE']\n",
        "            if analysis.get('ticker') not in valid_tickers:\n",
        "                print(f\"  ⚠ Ungültiger Ticker: {analysis.get('ticker')}\")\n",
        "                return None\n",
        "            \n",
        "            return analysis\n",
        "        else:\n",
        "            print(f\"  ⚠ Kein JSON in Antwort gefunden\")\n",
        "            return None\n",
        "            \n",
        "    except json.JSONDecodeError as e:\n",
        "        print(f\"  ⚠ JSON Parse Error: {e}\")\n",
        "        return None\n",
        "    except Exception as e:\n",
        "        print(f\"  ⚠ Fehler bei Gemini-Analyse: {e}\")\n",
        "        return None\n",
        "\n",
        "print(\"✅ analyze_article_with_gemini() definiert\")"
      ]
    },
    {
      "cell_type": "markdown",
      "metadata": {},
      "source": [
        "## 5️⃣ Datenverarbeitung"
      ]
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {},
      "outputs": [],
      "source": [
        "def parse_timeframe(timeframe_str, publish_date):\n",
        "    \"\"\"\n",
        "    Wandelt Zeitfenster-String in Start/End Dates um.\n",
        "    Beispiele: \"0-3d\", \"-1d-5d\", \"0-6h\", \"0-30d\"\n",
        "    \"\"\"\n",
        "    try:\n",
        "        # Parse Format: [START][UNIT]-[END][UNIT]\n",
        "        match = re.match(r'(-?\\d+)([dhm]?)-(\\d+)([dhm])', timeframe_str)\n",
        "        if not match:\n",
        "            print(f\"  ⚠ Ungültiges Zeitfenster-Format: {timeframe_str}\")\n",
        "            return None, None\n",
        "        \n",
        "        start_val = int(match.group(1))\n",
        "        start_unit = match.group(2) or 'd'\n",
        "        end_val = int(match.group(3))\n",
        "        end_unit = match.group(4) or 'd'\n",
        "        \n",
        "        # Konvertiere zu Timedeltas\n",
        "        unit_map = {'d': 'days', 'h': 'hours', 'm': 'minutes'}\n",
        "        \n",
        "        start_delta = timedelta(**{unit_map[start_unit]: start_val})\n",
        "        end_delta = timedelta(**{unit_map[end_unit]: end_val})\n",
        "        \n",
        "        start_date = publish_date + start_delta\n",
        "        end_date = publish_date + end_delta\n",
        "        \n",
        "        # Runde auf volle Tage\n",
        "        start_date = start_date.replace(hour=0, minute=0, second=0, microsecond=0)\n",
        "        end_date = end_date.replace(hour=23, minute=59, second=59, microsecond=999999)\n",
        "        \n",
        "        return start_date, end_date\n",
        "        \n",
        "    except Exception as e:\n",
        "        print(f\"  ⚠ Fehler beim Parsen von Zeitfenster '{timeframe_str}': {e}\")\n",
        "        return None, None\n",
        "\n",
        "print(\"✅ parse_timeframe() definiert\")"
      ]
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {},
      "outputs": [],
      "source": [
        "def get_stock_data_for_timeframe(stocks_data, ticker, start_date, end_date):\n",
        "    \"\"\"Extrahiert Aktiendaten für ein Zeitfenster\"\"\"\n",
        "    if ticker not in stocks_data:\n",
        "        return None\n",
        "    \n",
        "    df = stocks_data[ticker]\n",
        "    mask = (df['Date'] >= start_date) & (df['Date'] <= end_date)\n",
        "    filtered = df[mask].copy()\n",
        "    \n",
        "    return filtered if len(filtered) > 0 else None\n",
        "\n",
        "print(\"✅ get_stock_data_for_timeframe() definiert\")"
      ]
    },
    {
      "cell_type": "markdown",
      "metadata": {},
      "source": [
        "## 6️⃣ Visualisierung"
      ]
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {},
      "outputs": [],
      "source": [
        "def create_visualization(article_data, stock_data, output_path):\n",
        "    \"\"\"Erstellt Visualisierung des Aktienkurses mit News-Kontext\"\"\"\n",
        "    fig, ax = plt.subplots(figsize=(14, 8))\n",
        "    \n",
        "    # Aktienkurs plotten\n",
        "    ax.plot(stock_data['Date'], stock_data['Close'], \n",
        "            label=f\"{article_data['ticker']} Schlusskurs\", \n",
        "            linewidth=2.5, color='#2E86AB', marker='o', markersize=4)\n",
        "    \n",
        "    # Veröffentlichungsdatum markieren\n",
        "    pub_date = article_data['published_date']\n",
        "    ax.axvline(x=pub_date, color='red', linestyle='--', linewidth=2.5, \n",
        "               label=f'📰 Veröffentlichung: {pub_date.strftime(\"%Y-%m-%d %H:%M\")}', alpha=0.8)\n",
        "    \n",
        "    # Zeitfenster hervorheben\n",
        "    ax.axvspan(article_data['start_date'], article_data['end_date'], \n",
        "               alpha=0.15, color='yellow', label='⏱️ Analyse-Zeitfenster')\n",
        "    \n",
        "    # Formatierung\n",
        "    ax.set_xlabel('Datum', fontsize=13, fontweight='bold')\n",
        "    ax.set_ylabel('Kurs (USD)', fontsize=13, fontweight='bold')\n",
        "    \n",
        "    title = f\"{article_data['ticker']}: {article_data['headline'][:90]}...\"\n",
        "    subtitle = f\"Sentiment: {article_data['sentiment']} | Zeitfenster: {article_data['timeframe']}\"\n",
        "    ax.set_title(f\"{title}\\n{subtitle}\", fontsize=12, pad=20)\n",
        "    \n",
        "    ax.legend(loc='best', fontsize=10)\n",
        "    ax.grid(True, alpha=0.3, linestyle=':', linewidth=0.7)\n",
        "    \n",
        "    # Datum-Formatierung\n",
        "    ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d'))\n",
        "    ax.xaxis.set_major_locator(mdates.AutoDateLocator())\n",
        "    plt.xticks(rotation=45, ha='right')\n",
        "    \n",
        "    # Statistiken als Text-Box\n",
        "    price_start = stock_data['Close'].iloc[0]\n",
        "    price_end = stock_data['Close'].iloc[-1]\n",
        "    price_change = price_end - price_start\n",
        "    price_change_pct = (price_change / price_start) * 100\n",
        "    \n",
        "    stats_text = f\"📊 Statistiken:\\n\"\n",
        "    stats_text += f\"Start: ${price_start:.2f}\\n\"\n",
        "    stats_text += f\"Ende: ${price_end:.2f}\\n\"\n",
        "    stats_text += f\"Änderung: ${price_change:.2f} ({price_change_pct:+.2f}%)\\n\\n\"\n",
        "    stats_text += f\"💡 Begründung:\\n{article_data['reasoning'][:200]}...\"\n",
        "    \n",
        "    ax.text(0.02, 0.98, stats_text, transform=ax.transAxes, \n",
        "            fontsize=9, verticalalignment='top',\n",
        "            bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.8, pad=1),\n",
        "            family='monospace')\n",
        "    \n",
        "    plt.tight_layout()\n",
        "    plt.savefig(output_path, dpi=150, bbox_inches='tight')\n",
        "    plt.close()\n",
        "    \n",
        "    return str(output_path)\n",
        "\n",
        "print(\"✅ create_visualization() definiert\")"
      ]
    },
    {
      "cell_type": "markdown",
      "metadata": {},
      "source": [
        "## 7️⃣ Hauptanalyse - Verarbeitung der Artikel"
      ]
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {},
      "outputs": [],
      "source": [
        "# Erste 20 Artikel auswählen\n",
        "articles_to_analyze = news_df.head(MAX_ARTICLES).copy()\n",
        "\n",
        "print(\"=\" * 80)\n",
        "print(f\"🚀 STARTE ANALYSE VON {len(articles_to_analyze)} ARTIKELN\")\n",
        "print(\"=\" * 80)\n",
        "print()"
      ]
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {},
      "outputs": [],
      "source": [
        "# Ergebnislisten\n",
        "results = []\n",
        "failed_articles = []\n",
        "\n",
        "# Artikel durchlaufen\n",
        "for idx, row in articles_to_analyze.iterrows():\n",
        "    print(f\"\\n{'='*80}\")\n",
        "    print(f\"📄 Artikel {idx+1}/{len(articles_to_analyze)}\")\n",
        "    print(f\"{'='*80}\")\n",
        "    \n",
        "    # Extrahiere Artikel-Daten (passe Spaltennamen an deine CSV an!)\n",
        "    headline = row.get('headline', row.get('title', 'Keine Headline'))\n",
        "    url = row.get('url', row.get('link', ''))\n",
        "    summary = row.get('summary', row.get('description', ''))\n",
        "    \n",
        "    # Datum parsen\n",
        "    pub_date_str = row.get('published', row.get('datetime', row.get('date', '')))\n",
        "    try:\n",
        "        pub_date = pd.to_datetime(pub_date_str)\n",
        "    except:\n",
        "        print(f\"⚠ Ungültiges Datum: {pub_date_str}, überspringe Artikel\")\n",
        "        failed_articles.append({\n",
        "            'index': idx, \n",
        "            'reason': 'invalid_date', \n",
        "            'headline': headline\n",
        "        })\n",
        "        continue\n",
        "    \n",
        "    print(f\"📰 Headline: {headline[:100]}...\")\n",
        "    print(f\"🔗 URL: {url}\")\n",
        "    print(f\"📅 Veröffentlicht: {pub_date}\")\n",
        "    print()\n",
        "    \n",
        "    # Vollständigen Artikel mit Gemini extrahieren\n",
        "    print(\"🤖 Extrahiere Artikel-Inhalt mit Gemini...\")\n",
        "    full_content = extract_article_with_gemini(url, headline, summary)\n",
        "    print(f\"✓ {len(full_content)} Zeichen extrahiert\")\n",
        "    print()\n",
        "    \n",
        "    # Mit Gemini analysieren\n",
        "    print(\"🧠 Analysiere mit Gemini...\")\n",
        "    analysis = analyze_article_with_gemini(headline, summary, full_content, pub_date)\n",
        "    \n",
        "    if not analysis or analysis['ticker'] == 'NONE':\n",
        "        print(\"⚠ Keine relevante Aktie identifiziert, überspringe Artikel\")\n",
        "        failed_articles.append({\n",
        "            'index': idx, \n",
        "            'reason': 'no_ticker', \n",
        "            'headline': headline\n",
        "        })\n",
        "        continue\n",
        "    \n",
        "    print(f\"✓ Analyse abgeschlossen:\")\n",
        "    print(f\"  🎯 Ticker: {analysis['ticker']}\")\n",
        "    print(f\"  ⏱️  Zeitfenster: {analysis['timeframe']}\")\n",
        "    print(f\"  😊 Sentiment: {analysis['sentiment']}\")\n",
        "    print(f\"  💡 Begründung: {analysis['reasoning'][:100]}...\")\n",
        "    print()\n",
        "    \n",
        "    # Zeitfenster parsen\n",
        "    start_date, end_date = parse_timeframe(analysis['timeframe'], pub_date)\n",
        "    if not start_date or not end_date:\n",
        "        print(\"⚠ Ungültiges Zeitfenster, überspringe Artikel\")\n",
        "        failed_articles.append({\n",
        "            'index': idx, \n",
        "            'reason': 'invalid_timeframe', \n",
        "            'headline': headline\n",
        "        })\n",
        "        continue\n",
        "    \n",
        "    print(f\"📅 Zeitfenster: {start_date.date()} bis {end_date.date()}\")\n",
        "    \n",
        "    # Aktiendaten für Zeitfenster abrufen\n",
        "    stock_data = get_stock_data_for_timeframe(stocks_data, analysis['ticker'], start_date, end_date)\n",
        "    \n",
        "    if stock_data is None or len(stock_data) == 0:\n",
        "        print(\"⚠ Keine Aktiendaten für Zeitfenster verfügbar, überspringe Artikel\")\n",
        "        failed_articles.append({\n",
        "            'index': idx, \n",
        "            'reason': 'no_stock_data', \n",
        "            'headline': headline\n",
        "        })\n",
        "        continue\n",
        "    \n",
        "    print(f\"✓ {len(stock_data)} Handelstage gefunden\")\n",
        "    print()\n",
        "    \n",
        "    # Visualisierung erstellen\n",
        "    vis_filename = OUTPUT_DIR / f\"artikel_{idx+1:02d}_{analysis['ticker']}.png\"\n",
        "    \n",
        "    article_result = {\n",
        "        'index': idx + 1,\n",
        "        'headline': headline,\n",
        "        'url': url,\n",
        "        'published_date': pub_date,\n",
        "        'ticker': analysis['ticker'],\n",
        "        'timeframe': analysis['timeframe'],\n",
        "        'start_date': start_date,\n",
        "        'end_date': end_date,\n",
        "        'sentiment': analysis['sentiment'],\n",
        "        'reasoning': analysis['reasoning'],\n",
        "        'visualization': str(vis_filename),\n",
        "        'stock_data_points': len(stock_data),\n",
        "        'price_start': float(stock_data['Close'].iloc[0]),\n",
        "        'price_end': float(stock_data['Close'].iloc[-1]),\n",
        "        'price_change': float(stock_data['Close'].iloc[-1] - stock_data['Close'].iloc[0]),\n",
        "        'price_change_pct': float((stock_data['Close'].iloc[-1] - stock_data['Close'].iloc[0]) / stock_data['Close'].iloc[0] * 100)\n",
        "    }\n",
        "    \n",
        "    print(\"📊 Erstelle Visualisierung...\")\n",
        "    create_visualization(article_result, stock_data, vis_filename)\n",
        "    \n",
        "    results.append(article_result)\n",
        "    \n",
        "    print(\"✅ Artikel erfolgreich verarbeitet!\")\n",
        "    \n",
        "    # Rate limiting (Gemini API hat Limits)\n",
        "    time.sleep(2)\n",
        "\n",
        "print(\"\\n\" + \"=\"*80)\n",
        "print(\"🎉 ANALYSE ABGESCHLOSSEN!\")\n",
        "print(\"=\"*80)"
      ]
    },
    {
      "cell_type": "markdown",
      "metadata": {},
      "source": [
        "## 8️⃣ Ergebnisse speichern"
      ]
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {},
      "outputs": [],
      "source": [
        "# CSV speichern\n",
        "results_df = pd.DataFrame(results)\n",
        "csv_path = OUTPUT_DIR / \"analysis_results.csv\"\n",
        "results_df.to_csv(csv_path, index=False)\n",
        "print(f\"✅ CSV gespeichert: {csv_path}\")\n",
        "\n",
        "# JSON speichern\n",
        "json_path = OUTPUT_DIR / \"analysis_results.json\"\n",
        "with open(json_path, 'w', encoding='utf-8') as f:\n",
        "    json.dump(results, f, indent=2, default=str)\n",
        "print(f\"✅ JSON gespeichert: {json_path}\")\n",
        "\n",
        "# Failed Articles speichern\n",
        "if failed_articles:\n",
        "    failed_path = OUTPUT_DIR / \"failed_articles.json\"\n",
        "    with open(failed_path, 'w', encoding='utf-8') as f:\n",
        "        json.dump(failed_articles, f, indent=2, default=str)\n",
        "    print(f\"⚠ Fehlgeschlagene Artikel: {failed_path}\")"
      ]
    },
    {
      "cell_type": "markdown",
      "metadata": {},
      "source": [
        "## 9️⃣ Zusammenfassung & Visualisierung der Ergebnisse"
      ]
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {},
      "outputs": [],
      "source": [
        "print(\"\\n\" + \"=\"*80)\n",
        "print(\"📊 ZUSAMMENFASSUNG\")\n",
        "print(\"=\"*80)\n",
        "print(f\"✅ Erfolgreich analysiert: {len(results)}/{MAX_ARTICLES}\")\n",
        "print(f\"❌ Fehlgeschlagen: {len(failed_articles)}/{MAX_ARTICLES}\")\n",
        "print()\n",
        "\n",
        "if len(results) > 0:\n",
        "    print(\"📈 Verteilung nach Ticker:\")\n",
        "    ticker_counts = results_df['ticker'].value_counts()\n",
        "    for ticker, count in ticker_counts.items():\n",
        "        print(f\"  {ticker}: {count} Artikel\")\n",
        "    print()\n",
        "    \n",
        "    print(\"😊 Verteilung nach Sentiment:\")\n",
        "    sentiment_counts = results_df['sentiment'].value_counts()\n",
        "    for sentiment, count in sentiment_counts.items():\n",
        "        print(f\"  {sentiment}: {count} Artikel\")\n",
        "    print()\n",
        "    \n",
        "    print(\"💰 Durchschnittliche Preisänderung:\")\n",
        "    avg_change = results_df['price_change_pct'].mean()\n",
        "    print(f\"  {avg_change:+.2f}%\")\n",
        "    print()\n",
        "\n",
        "print(f\"📁 Alle Ergebnisse in: {OUTPUT_DIR.absolute()}\")\n",
        "print(\"=\"*80)"
      ]
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {},
      "outputs": [],
      "source": [
        "# Zeige Ergebnistabelle\n",
        "if len(results) > 0:\n",
        "    display_cols = ['index', 'ticker', 'sentiment', 'timeframe', 'price_change_pct', 'headline']\n",
        "    display_df = results_df[display_cols].copy()\n",
        "    display_df['headline'] = display_df['headline'].str[:60] + '...'\n",
        "    display_df['price_change_pct'] = display_df['price_change_pct'].round(2)\n",
        "    \n",
        "    print(\"\\n📋 Ergebnistabelle:\")\n",
        "    display(display_df)"
      ]
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {},
      "outputs": [],
      "source": [
        "# Zeige erste 3 Visualisierungen\n",
        "print(\"\\n🖼️ Erste 3 Visualisierungen:\\n\")\n",
        "for i, result in enumerate(results[:3]):\n",
        "    print(f\"\\n{'='*80}\")\n",
        "    print(f\"Artikel {i+1}: {result['ticker']} - {result['headline'][:70]}...\")\n",
        "    print(f\"{'='*80}\")\n",
        "    display(Image(filename=result['visualization']))"
      ]
    },
    {
      "cell_type": "markdown",
      "metadata": {},
      "source": [
        "## 🎯 Fertig!\n",
        "\n",
        "Die Analyse ist abgeschlossen. Alle Ergebnisse findest du im `analysis_results/` Ordner:\n",
        "\n",
        "- **CSV**: `analysis_results.csv` - Tabellarische Übersicht\n",
        "- **JSON**: `analysis_results.json` - Strukturierte Daten\n",
        "- **Visualisierungen**: `artikel_XX_TICKER.png` - Charts für jeden Artikel\n",
        "\n",
        "Du kannst dieses Notebook beliebig erweitern oder anpassen!"
      ]
    }
  ],
  "metadata": {
    "kernelspec": {
      "display_name": "Python 3",
      "language": "python",
      "name": "python3"
    },
    "language_info": {
      "codemirror_mode": {
        "name": "ipython",
        "version": 3
      },
      "file_extension": ".py",
      "mimetype": "text/x-python",
      "name": "python",
      "nbconvert_exporter": "python",
      "pygments_lexer": "ipython3",
      "version": "3.10.0"
    }
  },
  "nbformat": 4,
  "nbformat_minor": 4
}

NameError: name 'null' is not defined